In [15]:
import warnings
warnings.filterwarnings("ignore")

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt       

from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer

In [2]:
data = pd.read_csv("data/recipes.csv")

In [3]:
data.head()

,RecipeId,Name,AuthorId,AuthorName,CookTime,PrepTime,TotalTime,DatePublished,Description,Images,...,SaturatedFatContent,CholesterolContent,SodiumContent,CarbohydrateContent,FiberContent,SugarContent,ProteinContent,RecipeServings,RecipeYield,RecipeInstructions
0,38,Low-Fat Berry Blue Frozen Dessert,1533,Dancer,PT24H,PT45M,PT24H45M,1999-08-09T21:46:00Z,Make and share this Low-Fat Berry Blue Frozen ...,"c(""https://img.sndimg.com/food/image/upload/w_...",...,1.3,8.0,29.8,37.1,3.6,30.2,3.2,4.0,NaN,"c(""Toss 2 cups berries with sugar."", ""Let stan..."
1,39,Biryani,1567,elly9812,PT25M,PT4H,PT4H25M,1999-08-29T13:12:00Z,Make and share this Biryani recipe from Food.com.,"c(""https://img.sndimg.com/food/image/upload/w_...",...,16.6,372.8,368.4,84.4,9.0,20.4,63.4,6.0,NaN,"c(""Soak saffron in warm milk for 5 minutes and..."
2,40,Best Lemonade,1566,Stephen Little,PT5M,PT30M,PT35M,1999-09-05T19:52:00Z,This is from one of my first Good House Keepi...,"c(""https://img.sndimg.com/food/image/upload/w_...",...,0.0,0.0,1.8,81.5,0.4,77.2,0.3,4.0,NaN,"c(""Into a 1 quart Jar with tight fitting lid, ..."
3,41,Carina's Tofu-Vegetable Kebabs,1586,Cyclopz,PT20M,PT24H,PT24H20M,1999-09-03T14:54:00Z,This dish is best prepared a day in advance to...,"c(""https://img.sndimg.com/food/image/upload/w_...",...,3.8,0.0,1558.6,64.2,17.3,32.1,29.3,2.0,4 kebabs,"c(""Drain the tofu, carefully squeezing out exc..."
4,42,Cabbage Soup,1538,Duckie067,PT30M,PT20M,PT50M,1999-09-19T06:19:00Z,Make and share this Cabbage Soup recipe from F...,"""https://img.sndimg.com/food/image/upload/w_55...",...,0.1,0.0,959.3,25.1,4.8,17.7,4.3,4.0,NaN,"c(""Mix everything together and bring to a boil..."


In [4]:
df = data.copy()
columns = [ "RecipeId", "Name", "CookTime", "PrepTime", "TotalTime", "RecipeIngredientParts",
            "Calories", "FatContent", "SaturatedFatContent", "CholesterolContent",
            "SodiumContent", "CarbohydrateContent", "FiberContent", "SugarContent", "ProteinContent",
            "RecipeInstructions" 
]
df = df[columns]

In [5]:
max_Calories = 2000
max_daily_Fat = 100
max_daily_Saturatedfat = 13
max_daily_Cholesterol = 300
max_daily_Sodium = 2300
max_daily_Carbohydrate = 325
max_daily_Fiber = 40
max_daily_Sugar = 40
max_daily_Protein = 200
max_list = [
            max_Calories, max_daily_Fat, max_daily_Saturatedfat, 
            max_daily_Cholesterol, max_daily_Sodium, 
            max_daily_Carbohydrate, max_daily_Fiber, max_daily_Sugar, max_daily_Protein
        ]


min_Calories = 0
min_daily_Fat = 0
min_daily_Saturatedfat = 0
min_daily_Cholesterol = 0
min_daily_Sodium = 0
min_daily_Carbohydrate = 0
min_daily_Fiber = 0
min_daily_Sugar = 0
min_daily_Protein = 0
min_list = [
            min_Calories, min_daily_Fat, min_daily_Saturatedfat, 
            min_daily_Cholesterol, min_daily_Sodium, 
            min_daily_Carbohydrate, min_daily_Fiber, min_daily_Sugar, min_daily_Protein
        ]

In [6]:
for column,maximum in zip(df.columns[6:15],max_list):
    df = df[df[column]<maximum]

In [7]:
df.iloc[:,6:15].corr()

,Calories,FatContent,SaturatedFatContent,CholesterolContent,SodiumContent,CarbohydrateContent,FiberContent,SugarContent,ProteinContent
Calories,1.000000,0.767356,0.603317,0.478934,0.501082,0.711640,0.458711,0.180895,0.689447
FatContent,0.767356,1.000000,0.767357,0.440515,0.381944,0.223549,0.192142,0.042603,0.468088
SaturatedFatContent,0.603317,0.767357,1.000000,0.512186,0.319671,0.176623,0.044003,0.090721,0.388618
CholesterolContent,0.478934,0.440515,0.512186,1.000000,0.335843,0.066104,-0.047346,-0.036112,0.675302
SodiumContent,0.501082,0.381944,0.319671,0.335843,1.000000,0.294636,0.260479,-0.055518,0.500457
CarbohydrateContent,0.711640,0.223549,0.176623,0.066104,0.294636,1.000000,0.580535,0.390120,0.255447
FiberContent,0.458711,0.192142,0.044003,-0.047346,0.260479,0.580535,1.000000,0.068758,0.273488
SugarContent,0.180895,0.042603,0.090721,-0.036112,-0.055518,0.390120,0.068758,1.000000,-0.120441
ProteinContent,0.689447,0.468088,0.388618,0.675302,0.500457,0.255447,0.273488,-0.120441,1.000000


In [8]:
scaler=StandardScaler()

prep = scaler.fit_transform(df.iloc[:,6:15].to_numpy())

In [9]:
neighbor = NearestNeighbors(metric="cosine", algorithm="brute")
neighbor.fit(prep)

NearestNeighbors(algorithm='brute', metric='cosine')

In [10]:
transformer = FunctionTransformer(neighbor.kneighbors,kw_args={"return_distance":False})
pipeline=Pipeline([("std_scaler",scaler),("NN",transformer)])

In [11]:
params={"n_neighbors":10, "return_distance":False}

pipeline.get_params()
pipeline.set_params(NN__kw_args=params)

Pipeline(steps=[('std_scaler', StandardScaler()),
                ('NN',
                 FunctionTransformer(func=<bound method KNeighborsMixin.kneighbors of NearestNeighbors(algorithm='brute', metric='cosine')>,
                                     kw_args={'n_neighbors': 10,
                                              'return_distance': False}))])

In [12]:
for column, minimum in zip(df.columns[6:15], min_list):
    df = df[df[column] >= minimum]
    
for column, maximum in zip(df.columns[6:15], max_list):
    df = df[df[column] < maximum]

# END TO END

In [13]:
def scaling(dataframe):
    scaler = StandardScaler()
    prep_data = scaler.fit_transform(dataframe.iloc[:, 6:15].to_numpy())
    return prep_data, scaler

# Function for nearest neighbors prediction
def nn_predictor(prep_data):
    neigh = NearestNeighbors(metric='cosine', algorithm='brute')
    neigh.fit(prep_data)
    return neigh

# Function to build pipeline
def build_pipeline(neigh, scaler, params):
    transformer = FunctionTransformer(neigh.kneighbors, kw_args=params)
    pipeline = Pipeline([('std_scaler', scaler), ('NN', transformer)])
    return pipeline

# Function to extract filtered data
# Function to extract filtered data
def extract_data(dataframe, ingredient_filter, min_nutritional_values, max_nutritional_values):
    extracted_data = dataframe.copy()

    # Apply minimum nutritional value filters
    for column, minimum in zip(extracted_data.columns[6:15], min_nutritional_values):
        extracted_data = extracted_data[extracted_data[column] >= minimum]

    # Apply maximum nutritional value filters
    for column, maximum in zip(extracted_data.columns[6:15], max_nutritional_values):
        extracted_data = extracted_data[extracted_data[column] < maximum] 

    # Apply ingredient filter if provided
    if ingredient_filter is not None:
        for ingredient in ingredient_filter:
            extracted_data = extracted_data[extracted_data['RecipeIngredientParts'].str.contains(ingredient, regex=False)] 

    # Check if extracted data is empty after filtering
    if extracted_data.empty:
        raise ValueError("No data left after filtering. Check your nutritional value or ingredient filters.")

    return extracted_data


# Function to apply the pipeline to input
def apply_pipeline(pipeline, _input, extracted_data):
    return extracted_data.iloc[pipeline.transform(_input)[0]]

# End-to-end recommendation function
def recommand(dataframe, _input, min_nutritional_values, max_nutritional_values, ingredient_filter=None, params={'return_distance': False}):
    # Extract filtered data
    extracted_data = extract_data(df, ingredient_filter, min_nutritional_values, max_nutritional_values)
    print(f"Extracted data shape: {extracted_data.shape}")  # Check the shape of the data after filtering
    
    # Scaling the data
    prep_data, scaler = scaling(extracted_data)
    print(f"Prepared data shape: {prep_data.shape}")  # Check if scaling is working
    
    # Train the model
    neigh = nn_predictor(prep_data)
    
    # Build pipeline
    pipeline = build_pipeline(neigh, scaler, params)
    
    # Apply pipeline to get recommended items
    return apply_pipeline(pipeline, _input, extracted_data)

In [14]:
test_input = df.iloc[0:1, 6:15].to_numpy()

min_nutritional_values = [0, 0, 0, 0, 0, 0, 0, 0, 0]
max_nutritional_values = [500, 100, 50, 50, 30, 10, 200, 10, 150]

# Test the recommendation function
try:
    recommended_items = recommand(df, 
                                  _input=test_input, 
                                  min_nutritional_values=min_nutritional_values, 
                                  max_nutritional_values=max_nutritional_values, 
                                  ingredient_filter=['sugar'], 
                                  params={'return_distance': False})
    print(recommended_items)
except ValueError as e:
    print(f"Error: {e}")


Extracted data shape: (4499, 16)
Prepared data shape: (4499, 9)
        RecipeId                                       Name CookTime PrepTime  \
442662    458993                  Mini Blueberry Lime Tarts      NaN    PT30M   
186765    195155           Just Like Kfc Coleslaw - Copycat    PT10M    PT30M   
320346    332527  No Bake Cookies Made With Chocolate Chips     PT1M    PT10M   
405554    420396                 Mini Ice Cream Cookie Cups    PT40M    PT20M   
432546    448567                               Carrot Halva    PT50M    PT20M   

       TotalTime                              RecipeIngredientParts  Calories  \
442662     PT30M  c("pomegranate", "sugar", "fresh lime juice", ...      53.9   
186765     PT40M  c("cabbage", "carrot", "onion", "Miracle Whip"...      55.5   
320346     PT11M  c("butter", "milk", "sugar", "chocolate chips"...      58.2   
405554      PT1H  c("walnuts", "chocolate chips", "vanilla ice c...      61.4   
432546   PT1H10M  c("carrot", "sugar", "rais